# Scrape Página Camara de Representantes 

####  Se  sacan las páginas web que llevan al perfil de cada uno de los Representantes a la Camara 

In [ ]:
def scrap_url_representantes():
    html = requests.get('http://www.camara.gov.co/index.php/representantes').text
    sopa = BeautifulSoup(html, 'lxml')
    r_rep_links = sopa.find_all('span', class_='namereplist')
    rep_prefix = 'http://www.camara.gov.co/index.php/representantes/' 
    rep_sufix = re.findall('href="/representantes/(.+?)" hreflang',str(r_rep_links))
    rep_url = [rep_prefix + x for x in rep_sufix]
    
    return rep_url



####  se aplica la función y se obtiene una lista con 174 urls 

In [ ]:
rep_urls = scrap_url_representantes()

## Algoritmo para extraer la información de cada Representante 

In [ ]:
def rep_info(url):
    try:
        html = requests.get(url).text
        sopa = BeautifulSoup(html, 'lxml')
    
        r_mombre = sopa.find_all('div', class_='field field--name-name field--type-string field--label-hidden field__item')
        nombre = re.findall('field__item">(.+?)</div>', str(r_mombre))[-1]
        
        rep_Info = sopa.find_all('div', class_='col-xs-6 col-sm-6 col-md-6 contatributosrep')
        
        r_rep_partido = sopa.find_all('span', class_='partpolit')
        rep_partido = re.findall('hreflang=\"es\">(.+?)</a>', str(r_rep_partido))[-1]
        
        rep_circuns = re.findall('field--label-hidden field__item\">(.+?)</div>', str(rep_Info))[-1]
        
        try:
            r_rep_votos = sopa.find_all('div', class_='field field--name-field-votos-obtenidos field--type-string field--label-visually_hidden')
            rep_votos = re.findall('field__item\">(.+?)</div>', str(r_rep_votos))[-1]
        except:
            rep_votos= 'NA'
        return{'congresista':nombre,'Partido':rep_partido,'circunscripción':rep_circuns,'votos':rep_votos}
    #retorna un diccionario con cada elemento 
    
    except:
        nombre = 'link dañado'
        rep_partido = 'link dañado'
        rep_circuns = 'link dañado'
        rep_votos = 'link dañado'
        
        return{'congresista':nombre,'Partido':rep_partido,'circunscripción':rep_circuns,'votos':rep_votos}

#### se aplica la función  y se genera la Base de datos 

In [ ]:
rep_Dicc = list(map(rep_info,rep_urls))

rep_DF = pd.DataFrame(rep_Dicc)
representantes_DF = pd.DataFrame(rep_DF[['congresista','Partido','Camara']])

Se obtiene una base de datos con la información de cada Representante:

- Nombre
- Partido
- Circunscripción 
- votos 